# Finetuning Google's Gemma with Custom Data

This notebook follows a [Tutorial on YouTube](https://www.youtube.com/watch?v=iOdFUJiB0Zc).

This code walks through setting up Google's Gemma model, along with custom data, for fine-tuning, as well as some queries and results.

## I. Preparing the Data